In [7]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(3,activation='softmax')(x) #final layer with softmax activation

In [3]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [4]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [5]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/content/gdrive/MyDrive/Colab Notebooks/transfer-learning-master/train/', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 197 images belonging to 3 classes.


In [6]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
6/6 [==============================] - 47s 6s/step - loss: 1.6512 - accuracy: 0.4817
Epoch 2/5
6/6 [==============================] - 20s 3s/step - loss: 0.2098 - accuracy: 0.9527
Epoch 3/5
6/6 [==============================] - 20s 4s/step - loss: 0.2072 - accuracy: 0.9481
Epoch 4/5
6/6 [==============================] - 20s 3s/step - loss: 0.0823 - accuracy: 0.9723
Epoch 5/5
6/6 [==============================] - 20s 3s/step - loss: 0.1123 - accuracy: 0.9630


**COME FARE TRANSFER LEARNING SU UNA RETE PERSONALIZZATA**

In [ ]:
#importa il modello che hai salvato:
from keras.models import load_model
model = load_model("model_path.h5")

In [ ]:
#oppure importa separatamente pesi e architettura
form keras.models import model_from_json
with open("path_to_json_file.json") as json_file:
    model = model_from_json(json_file.read())
    model.load_weights("path_to_weights_file.h5")

In [ ]:
#freeze
for i in range(5):
    model.layers[i].trainable = False

In [ ]:
#fine tuning
for i in range(5,8):
    model.layers[i].trainable = True

In [ ]:
#aggiungi nuovi layer
ll = model.layers[8].output
ll = Dense(32)(ll)
ll = Dense(64)(ll)
ll = Dense(num_classes,activation="softmax")(ll)

new_model = Model(inputs=model.input,outputs=ll)